In [1]:
import pandas as pd

In [2]:
df= pd.read_csv("products.csv")

In [3]:
df

,Unnamed: 0,order_id,invoice_date,quantity,product_id,buyer_id,entity_name,product_name
0,0,18057,2021-04-29 13:21:47+04:00,5.0,13246,53915,Highway311,Garlic
1,1,18057,2021-04-29 13:21:47+04:00,3.0,13364,53915,Highway311,White Button Mushroom
2,2,18057,2021-04-29 13:21:47+04:00,5.0,13661,53915,Highway311,Cherry Tomato - Red
3,3,18057,2021-04-29 13:21:47+04:00,0.3,13117,53915,Highway311,Fresh Baby Spinach
4,4,18058,2021-04-29 10:22:49+04:00,7.0,12100,53998,Kitopi Catering Services LLC-CK-AQZ03,Fresh Chicken Breast - 500gms
...,...,...,...,...,...,...,...,...
634233,634233,209575,2024-09-21 08:14:44.832000+04:00,8.0,13239,50191,Andaz Dubai The Palm,Lettuce Romaine
634234,634234,209994,2024-09-23 08:59:17.045000+04:00,19.6,13901,53845,Origami Restautant LLC,Frozen Chicken 700g
634235,634235,210001,2024-09-23 11:22:14.087000+04:00,12.0,14482,54081,WALDORF ASTORIA HOTEL DUBAI PALM JUMEIRAH,Fresh Tofu
634236,634236,209990,2024-09-23 13:09:29.373000+04:00,19.6,13901,53845,Origami Restautant LLC,Frozen Chicken 700g


In [4]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


# Cleaning the data 

In [5]:

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Ensure `quantity` and `product_id` are numeric
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df['product_id'] = pd.to_numeric(df['product_id'], errors='coerce')

# Convert `invoice_date` to datetime
df['invoice_date'] = pd.to_datetime(df['invoice_date'], errors='coerce')

# Drop any rows with missing values
df = df.dropna()


start_date = '2024-01-01'
end_date = '2024-6-30'
df_filtered = df[(df['invoice_date'] >= start_date) & (df['invoice_date'] <= end_date)]

# Display the cleaned and filtered DataFrame
print("Cleaned and Filtered DataFrame:")
print(df_filtered.head())


Cleaned and Filtered DataFrame:
        Unnamed: 0  order_id              invoice_date  quantity  product_id  \
12405        12405    168962 2024-02-24 01:06:45+04:00       1.0       13785   
55312        55312    168413 2024-02-22 04:33:06+04:00       3.0       13372   
181676      181676    158109 2024-01-05 07:57:28+04:00       4.0       13718   
181677      181677    158109 2024-01-05 07:57:28+04:00       1.0       13236   
181678      181678    158109 2024-01-05 07:57:28+04:00       3.0       13574   

        buyer_id                                        entity_name  \
12405      54172                    PARK HOSPITALITY INVESTMENT LLC   
55312      54346  Sofitel Dubai The Obelisk (BR of Wafi Hotels LLC)   
181676     54688                       S M A K Foods Restaurant LLC   
181677     54688                       S M A K Foods Restaurant LLC   
181678     54688                       S M A K Foods Restaurant LLC   

                   product_name  
12405   Heirloom Mixed Tom

Model Building

In [6]:
import pandas as pd
from datetime import datetime, timedelta
from mlxtend.preprocessing import TransactionEncoder

# Convert all datetime columns to timezone-naive
df['invoice_date'] = df['invoice_date'].dt.tz_localize(None)

# Add month and year columns based on invoice_date
df['month'] = df['invoice_date'].dt.month
df['year'] = df['invoice_date'].dt.year

# Function to filter data for the last 6 months
def filter_last_6_months(df):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=6*30)  # Approximate 6 months
    return df[df['invoice_date'] >= start_date]

# Function to get unique products for all enterprises
def get_unique_products(df):
    df_filtered = filter_last_6_months(df)
    enterprises = df_filtered['entity_name'].unique()
    
    unique_products_dict = {}
    
    for enterprise in enterprises:
        df_enterprise = df_filtered[df_filtered['entity_name'] == enterprise]
        products = df_enterprise['product_name'].unique()
        unique_products_dict[enterprise] = products.tolist()
    
    return unique_products_dict

# Function to generate co-occurrence matrix
def generate_co_occurrence_matrix(df, enterprise_name, unique_products, min_confidence=0.2):
    # Filter transactions for the specific enterprise
    df_filtered = df[df['entity_name'] == enterprise_name]
    transactions = df_filtered.groupby('order_id')['product_name'].apply(list).tolist()
    
    # Create Transaction Encoder and DataFrame
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    transaction_df = pd.DataFrame(te_ary, columns=te.columns_)
    
    # Ensure unique_products are present in transaction_df columns
    valid_products = [product for product in unique_products if product in transaction_df.columns]
    
    # If no valid products are found, return an empty DataFrame
    if not valid_products:
        return pd.DataFrame(0.0, index=unique_products, columns=unique_products, dtype=float)
    
    # Filter DataFrame to include only valid products
    transaction_df = transaction_df[valid_products]
    
    # Calculate support
    item_support = transaction_df.sum(axis=0)
    co_occurrence_matrix = pd.DataFrame(0.0, index=valid_products, columns=valid_products, dtype=float)

    # Calculate co-occurrence counts
    for products in transactions:
        for product_a in products:
            for product_b in products:
                if product_a != product_b and product_a in valid_products and product_b in valid_products:
                    co_occurrence_matrix.loc[product_a, product_b] += 1
    
    # Calculate confidence scores
    for product_a in valid_products:
        for product_b in valid_products:
            if product_a != product_b:
                support_a = item_support[product_a]
                if support_a > 0:
                    co_occurrence_matrix.loc[product_a, product_b] = round(co_occurrence_matrix.loc[product_a, product_b] / support_a, 2)
                else:
                    co_occurrence_matrix.loc[product_a, product_b] = 0.0
    
    # Filter out recommendations below the minimum confidence threshold
    co_occurrence_matrix = co_occurrence_matrix.applymap(lambda x: x if x >= min_confidence else 0)
    return co_occurrence_matrix

# Function to calculate expected quantity
def calculate_expected_quantity(df, enterprise_id, product_name):
    df_filtered = filter_last_6_months(df)
    df_enterprise_product = df_filtered[(df_filtered['buyer_id'] == enterprise_id) & 
                                        (df_filtered['product_name'] == product_name)]
    
    last_month = datetime.now() - timedelta(days=30)
    df_last_month = df_filtered[(df_filtered['buyer_id'] == enterprise_id) &
                                (df_filtered['product_name'] == product_name) &
                                (df_filtered['invoice_date'] >= last_month)]
    
    if not df_last_month.empty:
        last_month_quantity = df_last_month['quantity'].sum()
        return last_month_quantity * 1.10  # 10% more than last month's quantity
    elif not df_enterprise_product.empty:
        quantities = df_enterprise_product['quantity'][df_enterprise_product['quantity'] != 0]
    else:
        df_product = df_filtered[df_filtered['product_name'] == product_name]
        quantities = df_product['quantity'][df_product['quantity'] != 0]
    
    return quantities.mean() if not quantities.empty else 0

# Function to get recommendations
def get_recommendations(df, min_confidence=0.2):
    unique_products_dict = get_unique_products(df)
    recommendations = []

    for enterprise, products in unique_products_dict.items():
        co_occurrence_matrix = generate_co_occurrence_matrix(df, enterprise, products, min_confidence)
        
        for product in co_occurrence_matrix.index:
            product_recommendations = co_occurrence_matrix.loc[product]
            avg_confidence = product_recommendations.mean()
            
            enterprise_id = df[df['entity_name'] == enterprise]['buyer_id'].iloc[0]
            recommended_product = product_recommendations.idxmax()
            product_id = df[df['product_name'] == recommended_product]['product_id'].iloc[0]
            
            bought_in_current_month = not df[(df['entity_name'] == enterprise) & 
                                             (df['product_name'] == recommended_product) & 
                                             (df['invoice_date'].dt.month == datetime.now().month)].empty
            
            if bought_in_current_month:
                confidence = "100.00%"
            elif avg_confidence > 0:
                confidence = f"{avg_confidence * 100:.2f}%"
            else:
                continue  # Skip recommendations with 0 confidence
            
            expected_quantity = calculate_expected_quantity(df, enterprise_id, recommended_product)
            
            recommendations.append({
                'Enterprise': enterprise,
                'Enterprise ID': enterprise_id,
                'Recommended Product': recommended_product,
                'Product ID': product_id,
                'Confidence': confidence,
                'Expected Quantity': expected_quantity
            })

    recommendations_df = pd.DataFrame(recommendations)
    return recommendations_df.drop_duplicates(subset=['Enterprise', 'Recommended Product'])

# Generate recommendations
recommendations_df = get_recommendations(df, min_confidence=0.2)

# Display the resulting DataFrame
print("Recommendations DataFrame:")
print(recommendations_df)


C:\Users\Pravin Kumar\AppData\Local\Temp\ipykernel_51084\2132780166.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invoice_date'] = df['invoice_date'].dt.tz_localize(None)
C:\Users\Pravin Kumar\AppData\Local\Temp\ipykernel_51084\2132780166.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['invoice_date'].dt.month
C:\Users\Pravin Kumar\AppData\Local\Temp\ipykernel_51084\2132780166.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

Recommendations DataFrame:
                                           Enterprise  Enterprise ID  \
0           WALDORF ASTORIA HOTEL DUBAI PALM JUMEIRAH          54081   
6           WALDORF ASTORIA HOTEL DUBAI PALM JUMEIRAH          54081   
7           WALDORF ASTORIA HOTEL DUBAI PALM JUMEIRAH          54081   
10                               ENCOUNTER COFFEE LLC          54430   
11                               ENCOUNTER COFFEE LLC          54430   
12    Dubai Festival City Shopping Centres Hotels LLC          54018   
14                                      Cash Customer          54103   
15                                      Cash Customer          54103   
17                                 Beach Rotana Hotel          54196   
19                             BLUEWATERS RESORTS LLC          54424   
20                             BLUEWATERS RESORTS LLC          54424   
21                                  Ramee Dream Hotel          54253   
22                    ARMED FORCES OF

In [7]:
recommendations_df

,Enterprise,Enterprise ID,Recommended Product,Product ID,Confidence,Expected Quantity
0,WALDORF ASTORIA HOTEL DUBAI PALM JUMEIRAH,54081,Round Tomato,13056,100.00%,35.00
6,WALDORF ASTORIA HOTEL DUBAI PALM JUMEIRAH,54081,White Button Mushroom,13364,100.00%,15.00
7,WALDORF ASTORIA HOTEL DUBAI PALM JUMEIRAH,54081,Lettuce Lollo Bionda,13524,100.00%,1.00
10,ENCOUNTER COFFEE LLC,54430,Pasteurized Gold Whole Egg,13794,100.00%,10.00
11,ENCOUNTER COFFEE LLC,54430,Halloumi Cheese Local 2.5kg Tub,13822,100.00%,5.00
12,Dubai Festival City Shopping Centres Hotels LLC,54018,Fresh Rosemary,13425,100.00%,0.25
14,Cash Customer,54103,Raspberry,13718,7.25%,1.00
15,Cash Customer,54103,Blackberry,13977,50.00%,1.00
17,Beach Rotana Hotel,54196,Labna Plain Full Fat 4kg,13421,100.00%,0.00
19,BLUEWATERS RESORTS LLC,54424,Cherry Tomato - Red,13661,50.00%,2.00


In [9]:
# Remove rows where 'Enterprise' is 'Cash Customer' 
filtered_df = recommendations_df[(recommendations_df['Enterprise'] != 'Cash Customer') & (recommendations_df['Product ID'] != 140945)]

# Add the 'created_date' column with the current date (no time)
filtered_df['created_date'] = datetime.now().date()

# Convert 'Confidence' column to float (removing the '%' sign first)
filtered_df['Confidence'] = filtered_df['Confidence'].str.replace('%', '').astype(float)

# Convert 'Bought in Current Month' column to boolean
filtered_df['Bought in Current Month'] = filtered_df['Bought in Current Month'].map({'Yes': True, 'No': False})

# Renaming the columns
filtered_df.rename(columns={
    'Enterprise': 'entity_name',
    'Enterprise ID': 'entity_id',
    'Recommended Product': 'product_name',
    'Product ID': 'product_id',
    'Confidence': 'confidence',
    'Expected Quantity': 'quantity',
    'Bought in Current Month': 'purchased',
    'created_date': 'created_date'
}, inplace=True)

# Display the first few rows of the filtered DataFrame
print(filtered_df.head())


                                  entity_name  entity_id  \
0   WALDORF ASTORIA HOTEL DUBAI PALM JUMEIRAH      54081   
6   WALDORF ASTORIA HOTEL DUBAI PALM JUMEIRAH      54081   
7   WALDORF ASTORIA HOTEL DUBAI PALM JUMEIRAH      54081   
10                       ENCOUNTER COFFEE LLC      54430   
11                       ENCOUNTER COFFEE LLC      54430   

                       product_name  product_id  confidence  quantity  \
0                      Round Tomato       13056       100.0      35.0   
6             White Button Mushroom       13364       100.0      15.0   
7              Lettuce Lollo Bionda       13524       100.0       1.0   
10       Pasteurized Gold Whole Egg       13794       100.0      10.0   
11  Halloumi Cheese Local 2.5kg Tub       13822       100.0       5.0   

    purchased created_date  
0       False   2024-09-23  
6       False   2024-09-23  
7       False   2024-09-23  
10      False   2024-09-23  
11      False   2024-09-23  


C:\Users\Pravin Kumar\AppData\Local\Temp\ipykernel_51084\599656731.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['created_date'] = datetime.now().date()
C:\Users\Pravin Kumar\AppData\Local\Temp\ipykernel_51084\599656731.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Confidence'] = filtered_df['Confidence'].str.replace('%', '').astype(float)
C:\Users\Pravin Kumar\AppData\Local\Temp\ipykernel_51084\599656731.py:11: SettingWithCopyWarning: 
A value is trying to be set on a c

In [10]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 0 to 53
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   entity_name   28 non-null     object 
 1   entity_id     28 non-null     int64  
 2   product_name  28 non-null     object 
 3   product_id    28 non-null     int64  
 4   confidence    28 non-null     float64
 5   quantity      28 non-null     float64
 6   purchased     28 non-null     bool   
 7   created_date  28 non-null     object 
dtypes: bool(1), float64(2), int64(2), object(3)
memory usage: 1.8+ KB


#Store Results in a new df

In [13]:
filtered_df.to_csv("best_recommendations_df.csv")